In [17]:
import matplotlib.pyplot as plt
import numpy as np 
import matplotlib.ticker as ticker
import matplotlib.patches as patches
import scipy
from scipy import special
import math
import scipy.fftpack
from numpy.fft import fft, ifft

In [2]:
def unpack(gh):
    data = []
    k, l = 0, 1
    while k + 1 < len(gh):
        for m in range(l + 1):
            if m == 0:
                data.append([l, m, gh[k], 0])
                k += 1
            else:
                data.append([l, m, gh[k], gh[k + 1]])
                k += 2
        l += 1
    return data

In [3]:
def f_AD_NAD(data,lmax):
    a=6371.2
    #c=a*0.53
    c=3485.0
    
    ratio=a/c
    
    Ls = list(range(2, lmax+1))#l=1 is skipped and calculated separetely in ad and P11
    #Ls = list(range(2, 9))#l=1 is skipped and calculated separetely in ad and P11, lmax=8 from Christensen et al. (2010) 
    
    ad=(2) * ((1e-3 * data[0][2])
                              ** 2 + (1e-3 * data[0][3])**2)
    P11= (2) * ((1e-3 * data[1][2])
                              ** 2 + (1e-3 * data[1][3])**2)
    
    #calculating Pnm for n=>2
    recno = 2# NEEDS to b 2 to skip g01,g11 and h11
    pow=0
    for l in Ls:
        for m in range(0, l + 1):
            pow += (ratio**(2*l-2)) * (l + 1) * ((1e-3 * data[recno][2])
                              ** 2 + (1e-3 * data[recno][3])**2)
            recno += 1
           
    
    #ad=1.0
    nad=P11+pow
    return ad/nad



In [4]:
def f_O_E(data,lmax):
    a=6371.2
    #c=a*0.53
    c=3485.0
    
    ratio=a/c
    
    Ls = list(range(2, lmax+1))#l=1 is skipped as defined in Christensen et al. (2010)
    #Ls = list(range(2, 9))#lmin=2 and Lmax=8 as defined in Christensen et al. (2010)
    
    #calculating Pnm for n=>2
    recno = 2# NEEDS to b 2 to skip g01,g11 and h11
    O=0
    E=0
    for l in Ls:
        for m in range(0, l + 1):
            if (l+m)%2 == 0 :
                E += (ratio**(2*l+4)) * (l + 1) * ((1e-3 * data[recno][2])
                                  ** 2 + (1e-3 * data[recno][3])**2)
                recno += 1
            else:
                O += (ratio**(2*l+4)) * (l + 1) * ((1e-3 * data[recno][2])
                                  ** 2 + (1e-3 * data[recno][3])**2)
                recno += 1
    
    return O/E



In [5]:
def f_Z_NZ(data,lmax):
    a=6371.2
    #c=a*0.53
    c=3485.0
    
    ratio=a/c
    
    Ls = list(range(2, lmax+1))#l=1 is skipped as defined in Christensen et al. (2010)
    #Ls = list(range(2, 9))#lmin=2 and Lmax=8 as defined in Christensen et al. (2010)
    
    #calculating Pnm for n=>2
    recno = 2# NEEDS to b 2 to skip g01,g11 and h11
    Z=0
    NZ=0
    for l in Ls:
        for m in range(0, l + 1):
            if m == 0 :
                Z += (ratio**(2*l-2)) * (l + 1) * ((1e-3 * data[recno][2])
                                  ** 2 + (1e-3 * data[recno][3])**2)
                recno += 1
            else:
                NZ += (ratio**(2*l-2)) * (l + 1) * ((1e-3 * data[recno][2])
                                  ** 2 + (1e-3 * data[recno][3])**2)
                recno += 1
    
    return Z/NZ

In [6]:
def f_FCF(Br,theta):
    degree2rad=np.pi/180
    Br2=np.sum(np.sin(theta*degree2rad)*(Br**2))/np.sum(np.sin(theta*degree2rad))
    Br4=np.sum(np.sin(theta*degree2rad)*(Br**4))/np.sum(np.sin(theta*degree2rad))
    
    return (Br4-(Br2**2))/(Br2**2)

In [7]:
def f_MIN_star(F,theta):    
    return np.min(F)/( f_mean(F,theta) )

In [8]:
def f_mean(F,theta):
    F=np.sum(np.sin(theta*0.0174533)*F)/np.sum(np.sin(theta*0.0174533))
    #print(F)
    return F

In [9]:
def f_PM(Br,theta):
    g01=1.0
    if (g01 >0):
        cond_NH = (theta <= 90)
        cond_SH = (theta >= 90)
        Br_NP = np.mean(Br[theta==0]) 
        Br_SP = np.mean(Br[theta==180])
      
        Br_z= np.absolute(Br).sum(axis=1) / Br.shape[1]

        #print(Br_z_SH)
        
        #print(Br_z.shape)
        #print(Br.shape)
        
        maxBr_SH=np.max(Br[cond_SH])
        minBr_NH=np.min(Br[cond_NH])
        maxBr_z = np.max(np.absolute(Br_z))
        
        
        NHPM=abs( Br_NP - minBr_NH ) / maxBr_z
        SHPM=abs( Br_SP - maxBr_SH ) / maxBr_z
        
    return NHPM,SHPM

In [10]:
def f_FPD(Br,lons_Br,lats_Br,lmax):
    
    Br=Br*(0.000001)
    Br2=(Br**2)*np.sin((90-lats_Br)*np.pi/180)
    zonal_ave=[]
    for e in np.arange(0,len(lons_Br[0]),1):
        a = np.sum(Br2[:,e])/np.sum(np.sin(90-lats_Br))
        zonal_ave.append(a)

    zonal_ave=np.asarray(zonal_ave)

    # sampling rate
    sr = len(zonal_ave)
    # sampling interval
    ts = 1.0/sr
    t = np.arange(0,1,ts)
    X = fft(zonal_ave)
    N = len(X)
    n = np.arange(N)
    T = N/sr
    freq = n/T 

    XX=np.abs(X)
    aux_sum=0
    for ell in np.arange(3,lmax+1,1):
        aux_sum+=XX[ell]

    FPD=((len(XX[3:lmax+1])+1)*XX[2])/(XX[1]+aux_sum)

    return FPD

In [11]:
def brfield(gsh,hsh,lmax,r,dlon=1,dlat=1):
    a=6371.2
    pi            = math.pi
    lats          = np.arange(-90,91,dlat)
    lons          = np.arange(-180,181,dlon)
    lons2d,lats2d = np.meshgrid(lons,lats)
    phi           = lons2d
    cost2d        = np.cos((90-lats2d)*pi/180)
    i             = 0
    br            = np.zeros((cost2d.shape))
    bphi          = np.zeros((cost2d.shape))
    btheta        = np.zeros((cost2d.shape))
    for l in np.arange(1,lmax+1,1):
        for m in np.arange(0,l+1,1):
            g=gsh[i]
            h=hsh[i]
            #calculate schmidt
            if m  == 0:
                schmidt = 1.0
            else:
                schmidt = ((-1.0)**m)*np.sqrt(2.0*math.factorial(l-m)/math.factorial(l+m))
            lagendre     = scipy.special.lpmv(m, l, cost2d)
            lagendre_    = scipy.special.lpmv(m, l-1, cost2d)
            div_lagendre = (l*cost2d*lagendre-(l+m)*lagendre_)/np.sqrt(1-cost2d**2)
            br      = br     + schmidt*(float(l)+1.0)*((a/r)**(l+2))*lagendre*(g*np.cos(float(m)*phi*pi/180.0)+h*np.sin(float(m)*phi*pi/180.0))
            i+=1
    return br,lons2d,lats2d

In [12]:
def magnetic_field(gsh,hsh,lmax,r,dlon=1,dlat=1):
    a=6371.2
    pi            = math.pi
    lats          = np.arange(-89,90,dlat)
    lons          = np.arange(-180,181,dlon)
    lons2d,lats2d = np.meshgrid(lons,lats)
    phi           = lons2d
    cost2d        = np.cos((90-lats2d)*pi/180)
    i             = 0
    br            = np.zeros((cost2d.shape))
    bphi          = np.zeros((cost2d.shape))
    btheta        = np.zeros((cost2d.shape))
    for l in np.arange(1,lmax+1,1):
        for m in np.arange(0,l+1,1):
            g=gsh[i]
            h=hsh[i]
            #calculate schmidt
            if m  == 0:
                schmidt = 1.0
            else:
                schmidt = ((-1.0)**m)*np.sqrt(2.0*math.factorial(l-m)/math.factorial(l+m))
            lagendre     = scipy.special.lpmv(m, l, cost2d)
            lagendre_    = scipy.special.lpmv(m, l-1, cost2d)
            div_lagendre = (l*cost2d*lagendre-(l+m)*lagendre_)/np.sqrt(1-cost2d**2)
            br           = br     + schmidt*(float(l)+1.0)*((a/r)**(l+2))*lagendre*(g*np.cos(float(m)*phi*pi/180.0)+h*np.sin(float(m)*phi*pi/180.0))
            bphi         = bphi   + schmidt*(m/np.sin((90-lats2d)*pi/180))*((a/r)**(l+2))*lagendre*(-g*np.sin(float(m)*phi*pi/180.0)+h*np.cos(float(m)*phi*pi/180.0))
            btheta       = btheta + schmidt*((a/r)**(l+2))*(div_lagendre)*(g*np.cos(float(m)*phi*pi/180.0)+h*np.sin(float(m)*phi*pi/180.0))
            i+=1
    return br,bphi,btheta,lons2d,lats2d

In [37]:
print("Model AD/NAD O/E Z/NZ FCF F*min PMM FPD PMD")
# MODELS=['acheo_mean4FTN',
#            'holo_mean4FTN','holo_median4FTN','holo_huber4FTN','holo_rw4FTN',
#            'pleis_mean4FTN','pleis_median4FTN','pleis_huber4FTN','pleis_rw4FTN']

MODELS=['archeo_huber4FTN.dat','archeo_median4FTN.dat','archeo_rw4FTN.dat','archeo_mean4FTN.dat',
        'holo_huber4FTN.dat','holo_median4FTN.dat','holo_rw4FTN.dat','holo_mean4FTN.dat',
        'pleis_huber4FTN.dat','pleis_median4FTN.dat','pleis_rw4FTN.dat','pleis_mean4FTN.dat']
for model in MODELS:
    sh = np.loadtxt("archeo_ref/%s" % model).T
    sh = sh.reshape(-1,35) #takes 35 spherical harmnics
    #print(sh.shape)
    a,b= sh.shape
    ad_nad = np.zeros((a))
    o_e = np.zeros((a))
    z_nz = np.zeros((a))
    fcf = np.zeros((a))
    f_min_star = np.zeros((a))
    pmm = np.zeros((a))
    fpd = np.zeros((a))
    pmd = np.zeros((a))
    i=0
    for t in np.arange(0,a,1):
        output_temp = open("temp.dat","w+")
        data=unpack(sh[t,:]) # unpack them into the form that lowes likes
        #print(data)
        data=np.array(data)
        g01=data[0][2]

        alt=-6371.2+3485
        ysize,xsize=data.shape
        lmax=5
        l_count=0
        for ll in range(0,lmax+1,1):
            l_count=l_count + (ll+1)

        for ii in range(0,l_count-1,1):
            output_temp.write( "%d %d %f %f\n" %(data[ii][0],data[ii][1],data[ii][2],data[ii][3]))

        output_temp.close()

        ls,ms,gs,hs= np.loadtxt("temp.dat").T
        
        z_nz[i]=f_Z_NZ(data,lmax)
        o_e[i]=f_O_E(data,lmax)
        ad_nad[i]=f_AD_NAD(data,lmax)

        r=6371.2
        Br,Bphi,Btheta,lons,lats=magnetic_field(gs,hs,lmax,r,dlon=2,dlat=2)
        Bs=np.sqrt(Bphi**2+Btheta**2+Br**2)
        f_min_star[i]=f_MIN_star(Bs,90-lats)
        r=+3485.0
        Br,lons,lats=brfield(gs,hs,lmax,r,dlon=2,dlat=2)
        fcf[i]=f_FCF(Br,90-lats)
        fpd[i]=f_FPD(Br,lons,lats,5)

        if(g01<0):
            NHPM,SHPM=f_PM(Br,90-lats)        
        else:
            NHPM,SHPM=f_PM(-Br,90-lats)

        pmm[i]=(NHPM+SHPM)/2
        pmd[i]=(NHPM-SHPM)
        i+=1
    print("%s %.3f(%.3f) %.3f(%.3f) %.3f(%.3f) %.3f(%.3f) %.3f(%.3f) %.3f(%.3f) %.3f(%.3f) %.3f(%.3f)\n" %(model,
                            np.mean(ad_nad),np.std(ad_nad),np.mean(o_e),np.std(o_e),
                            np.mean(z_nz),np.std(z_nz),np.mean(fcf),np.std(fcf),np.mean(f_min_star),np.std(f_min_star),
                            np.mean(pmm),np.std(pmm),np.mean(fpd),np.std(fpd),np.mean(pmd),np.std(pmd)))




Model AD/NAD O/E Z/NZ FCF F*min PMM FPD PMD


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


archeo_huber4FTN.dat 5.767(1.791) 0.787(0.291) 0.304(0.154) 1.116(0.162) 0.638(0.024) 0.236(0.113) 1.449(0.581) 0.281(0.193)

archeo_median4FTN.dat 5.755(1.858) 0.785(0.318) 0.309(0.169) 1.121(0.151) 0.637(0.022) 0.244(0.117) 1.478(0.586) 0.254(0.225)

archeo_rw4FTN.dat 5.674(1.751) 0.764(0.291) 0.335(0.165) 1.085(0.156) 0.638(0.024) 0.226(0.102) 1.453(0.548) 0.265(0.201)

archeo_mean4FTN.dat 5.881(1.867) 0.784(0.296) 0.312(0.158) 1.106(0.161) 0.638(0.024) 0.228(0.107) 1.449(0.564) 0.261(0.200)

holo_huber4FTN.dat 8.594(3.119) 1.081(0.556) 0.255(0.182) 1.192(0.230) 0.654(0.033) 0.283(0.128) 2.013(1.297) 0.152(0.309)

holo_median4FTN.dat 10.157(7.705) 0.995(0.506) 0.246(0.193) 1.185(0.232) 0.654(0.037) 0.265(0.133) 2.034(1.151) 0.127(0.328)

holo_rw4FTN.dat 12.593(5.856) 1.065(0.630) 0.206(0.170) 1.105(0.161) 0.653(0.031) 0.192(0.094) 2.410(1.660) 0.114(0.220)

holo_mean4FTN.dat 11.038(5.070) 1.040(0.606) 0.227(0.189) 1.131(0.190) 0.655(0.031) 0.219(0.098) 2.294(1.620) 0.130(0.252)

ple